In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from visualize import PyramidVisualizer, get_dataset, display_image, simple_visualizer
from metroem.aligner import Aligner
from corgie import residuals

In [2]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
#import torch; assert(torch.cuda.is_available())

In [4]:
data_dir = "/usr/people/popovych/datasets/fly_m4_paper_big2"
dataset_mip = 4
stage = 0
checkpoint_name = 'test_no_batchnorm'
#checkpoint_name = 'emb_nobs_x1'
#checkpoint_name = 'emb_x0'
#checkpoint_name = 'emb_x2_07_32'
crop_mode = 'middle'
checkpoint_folder = "

In [4]:
gs://tmacrina-corgie-test/models/fly_m4_210722_plain100_metric257/2_mip4in_mip4module

In [7]:
data_dir = "/usr/people/popovych/datasets/fly_m4_paper_big2"
dataset_mip = 4
stage = 0
checkpoint_name = 'test_no_batchnorm'
#checkpoint_name = 'emb_nobs_x1'
#checkpoint_name = 'emb_x0'
#checkpoint_name = 'emb_x2_07_32'
crop_mode = 'middle'
checkpoint_folder = "/usr/people/popovych/models/tommy_new/model"
#checkpoint_folder = "/usr/people/popovych/models/fly_emb_x0/0_mip4in_mip5module/model"

pyramid = Aligner(checkpoint_folder,
                    checkpoint_name=checkpoint_name,
                    train=False,
                    finetune_lr=3e-1,
                    finetune_sm=30e0,
                    finetune_iter=400,
                    finetune=False).cuda()

dataset = get_dataset(
    data_dir=data_dir, 
    dataset_mip=dataset_mip, 
    stage=stage, 
    checkpoint_name=checkpoint_name, 
    crop_mode=crop_mode,
    cropped_size=1024

)

viz = PyramidVisualizer(
    pyramid=pyramid, 
    dataset_mip=dataset_mip, 
    dataset=dataset
)

/usr/people/popovych/models/tommy_new/model/model_spec.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
identity.json
embedder_m0_3x3_fms3x.json
block_5convs_7x7_fms1to3_skip14.json
block_3convs_3x3_fms3to32to3.json
average_pool.json
embedder_fms4to3.json
block_3convs_3x3_fms4to32to3.json
average_pool.json
block_3convs_3x3_fms3to32to3.json
identity.json
identity.json
identity.json
upsample_residuals.json
identity.json
Adding 'x138000_y53000_z152' dataset.
Adding 'x135000_y50000_z152' dataset.
Loading file '/usr/people/popovych/datasets/fly_m4_paper_big2/x138000_y53000_z152_MIP4.h5...'
Loading file '/usr/people/popovych/datasets/fly_m4_paper_big2/x135000_y50000_z152_MIP4.h5...'


/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/artificery/parsers/categorical_regression/parse.py:38: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  requires_grad=False) - self.component_channels//2


In [8]:
viz.visualize(section_count=1, default_slice=10, default_x=0, default_y=0)

interactive(children=(Checkbox(value=False, description='Take from validation set'), Checkbox(value=True, desc…

In [5]:

from torch.autograd import Variable
import torch

def jacob(fields):
    def dx(f):
        p = Variable(torch.zeros((f.size(0),1,f.size(1),2), device='cuda'))
        return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
    def dy(f):
        p = Variable(torch.zeros((f.size(0),f.size(1),1,2), device='cuda'))
        return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
    fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
    field = torch.sum(torch.cat(fields, -1) ** 2, -1)
    return field


def lap(fields, device='cuda'):
    def dx(f):
        p = Variable(torch.zeros((f.size(0),1,f.size(1),2), device=device))
        return torch.cat((p, f[:,1:-1,:,:] - f[:,:-2,:,:], p), 1)
    def dy(f):
        p = Variable(torch.zeros((f.size(0),f.size(1),1,2), device=device))
        return torch.cat((p, f[:,:,1:-1,:] - f[:,:,:-2,:], p), 2)
    def dxf(f):
        p = Variable(torch.zeros((f.size(0),1,f.size(1),2), device=device))
        return torch.cat((p, f[:,1:-1,:,:] - f[:,2:,:,:], p), 1)
    def dyf(f):
        p = Variable(torch.zeros((f.size(0),f.size(1),1,2), device=device))
        return torch.cat((p, f[:,:,1:-1,:] - f[:,:,2:,:], p), 2)
    fields = map(lambda f: [dx(f), dy(f), dxf(f), dyf(f)], fields)
    fields = map(lambda fl: (sum(fl) / 4.0) ** 2, fields)
    field = sum(map(lambda f: torch.sum(f, -1), fields))
    return field

def jacob(fields):
    def dx(f):
        p = Variable(torch.zeros((f.size(0),1,f.size(1),2), device='cuda'))
        return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
    def dy(f):
        p = Variable(torch.zeros((f.size(0),f.size(1),1,2), device='cuda'))
        return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
    fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
    field = torch.sum(torch.cat(fields, -1) ** 2, -1)
    return field



def cjacob(fields):
    def center(f):
        fmean_x, fmean_y = torch.mean(f[:,:,:,0]).item(), torch.mean(f[:,:,:,1]).item()
        fmean = torch.cat((fmean_x * torch.ones((1,f.size(1), f.size(2),1), device='cuda'), fmean_y * torch.ones((1,f.size(1), f.size(2),1), device='cuda')), 3)
        fmean = Variable(fmean).cuda()
        return f - fmean

    def dx(f):
        p = Variable(torch.zeros((f.size(0),1,f.size(1),2), device='cuda'))
        d = torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
        return center(d)
    def dy(f):
        p = Variable(torch.zeros((f.size(0),f.size(1),1,2), device='cuda'))
        d = torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
        return center(d)

    fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
    field = torch.sum(torch.cat(fields, -1) ** 2, -1)
    return field

In [8]:
res.max()

NameError: name 'res' is not defined

In [3]:
import metroem
res = viz.run_sample['pred_res']
mask = (viz.run_sample['tgt'] != 0) * (viz.run_sample['pred_tgt'] != 0)
metroem.loss.rigidity(res)[mask.squeeze()].mean()

NameError: name 'viz' is not defined

In [79]:
jacob([res.permute(0, 2, 3, 1)]).squeeze()[mask.squeeze()].mean()

tensor(0.0077, device='cuda:0')

In [75]:
jacob([res.permute(0, 2, 3, 1)]).squeeze()[mask.squeeze()].mean()

tensor(0.0073, device='cuda:0')

In [23]:

metroem.loss.lap(viz.run_sample['pred_res']).mean()

AttributeError: module 'metroem.loss' has no attribute 'lap'

In [15]:
res = viz.vect_dic['Predicted Residual']

AttributeError: 'PyramidVisualizer' object has no attribute 'vect_dic'

In [10]:
i = 1
tgt_embs = pyramid.get_embeddings(dataset[i]['tgt'], level=0)
src_embs = pyramid.get_embeddings(dataset[i]['src'], level=0)

v = 0.6
vmin, vmax = -v, v
lx = 350
ly = 2
hx = 900
hy = 550
imgs = [dataset[i]['tgt'], dataset[i]['src'], residuals.shift_by_int(tgt_embs[1], 0, 0),
        residuals.shift_by_int(src_embs[1], 0, 0),
        residuals.shift_by_int(src_embs[1], 0, 1),
        residuals.shift_by_int(src_embs[1], 1, 0)
       ]
imgs = [img[lx:hx, ly:hy] for img in imgs]
simple_visualizer().visualize(imgs)

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3, 4, 5), value=0), IntText(value=…

In [ ]:
i = 17
tgt_embs = pyramid.get_embeddings(dataset[i]['tgt'], level=0)
src_embs = pyramid.get_embeddings(dataset[i]['src'], level=0)
lx = 400
ly = 200
hx = 700
hy = 500
imgs = [dataset[i]['tgt'], dataset[i]['src'], residuals.shift_by_int(tgt_embs[1], 0, 0), residuals.shift_by_int(src_embs[1], 0, 0)]
imgs = [img[lx:hx, ly:hy] for img in imgs]
simple_visualizer().visualize(imgs)

In [14]:
# good image
i = 16
tgt_embs = pyramid.get_embeddings(dataset[i]['tgt'], level=0)
src_embs = pyramid.get_embeddings(dataset[i]['src'], level=0)

v = 0.6
vmin, vmax = -v, v
lx = 350
ly = 200

hx = 900
hy = 750
imgs = [dataset[i]['tgt'], dataset[i]['src'], residuals.shift_by_int(tgt_embs[1], 0, 0), residuals.shift_by_int(src_embs[1], 0, 0)]
imgs = [img[lx:hx, ly:hy] for img in imgs]
simple_visualizer().visualize(imgs)

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [35]:
i = 1
embs = pyramid.get_embeddings(dataset[i]['tgt'], level=0)
print(embs[0].min(), embs[0].max())
v = 0.6
vmin, vmax = -v, v
imgs = [dataset[i]['tgt'], embs[0], emb_old]
simple_visualizer().visualize(imgs)

tensor(-0.0250, device='cuda:0') tensor(0.0357, device='cuda:0')


NameError: name 'emb_old' is not defined

In [ ]:
emb_old = embs[0]